In [93]:
import csv
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [94]:
cats_path_gray = 'cats_gray'
dogs_path_gray = 'dogs_gray'

In [95]:
os.listdir(cats_path_gray)[:2]

['cat1.png', 'cat10.png']

In [96]:
class Labels:
    DOG= "DOG"
    CAT = 'CAT'

### Labling cats

In [97]:
data = []
target = []
for img in os.listdir(cats_path_gray):
    image = cv2.imread(cats_path_gray+'/'+img, -1)
    image_as_array = np.array(image)
    data.append(image_as_array)
    target.append(Labels.CAT)

## Labling Dogs

In [98]:
for img in os.listdir(dogs_path_gray):
    image = cv2.imread(dogs_path_gray+'/'+img, -1)
    image_as_array = np.array(image)
    data.append(image_as_array)
    target.append(Labels.DOG)

In [29]:
data[-2:], target[-2:]

([array([[107, 127, 122, ..., 131, 145, 148],
         [112, 123, 117, ..., 131, 139, 141],
         [120, 117, 111, ..., 132, 131, 131],
         ...,
         [199, 200, 201, ..., 217, 218, 218],
         [199, 196, 191, ..., 218, 218, 218],
         [199, 192, 184, ..., 219, 219, 219]], dtype=uint8),
  array([[224, 233, 239, ..., 171, 177, 171],
         [223, 231, 237, ..., 164, 164, 158],
         [221, 229, 235, ..., 152, 142, 137],
         ...,
         [174, 180, 178, ..., 157, 166, 167],
         [179, 182, 176, ..., 161, 164, 166],
         [182, 183, 175, ..., 163, 163, 165]], dtype=uint8)],
 ['DOG', 'DOG'])

In [99]:
target.count(Labels.DOG), target.count(Labels.CAT)

(100, 100)

In [110]:
target_binary = []
for t in target:
    target_binary.append(0 if t == Labels.CAT else 1)

> **0** will represent a cat and **1** will represent a dog.
> Target Names are `['CAT', 'DOG']`

> **THE ARRAYS OF IMAGES SHOULD BE RESHAPED INTO A 1-D ARRAY**

In [111]:
reshaped_data = []
for d in data:
    reshaped_data.append(np.array(d).reshape(-1))

In [112]:
reshaped_data[:2], target[:2]

([array([40, 42, 45, ..., 72, 71, 71], dtype=uint8),
  array([14, 15, 16, ..., 39, 46, 48], dtype=uint8)],
 ['CAT', 'CAT'])

In [113]:
class CatDog:
    def __init__(self, reshaped_data, target, target_names):
        self.data = reshaped_data
        self.target = target
        self.target_names = target_names
        

In [116]:
np.array(reshaped_data).shape

(200, 40000)

In [117]:
load_cats_dog = CatDog(np.array(reshaped_data), np.array(target_binary),np.array([Labels.CAT, Labels.DOG]))

In [124]:
load_cats_dog.data[0],load_cats_dog.target_names, load_cats_dog.target.shape

(array([40, 42, 45, ..., 72, 71, 71], dtype=uint8),
 array(['CAT', 'DOG'], dtype='<U3'),
 (200,))

### Tranning a model

In [125]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score

In [126]:
load_cats_dog.data[0]

array([40, 42, 45, ..., 72, 71, 71], dtype=uint8)

In [128]:
X = load_cats_dog.data
y = load_cats_dog.target
X.shape, y.shape

((200, 40000), (200,))

In [129]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29, test_size= .25)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((150, 40000), (50, 40000), (150,), (50,))

In [130]:
model =  SVC(kernel='linear')
model

SVC(kernel='linear')

In [131]:
model.fit(X_train, y_train)

SVC(kernel='linear')

### Making Predictions

In [135]:
X_train[0], y_train[0]

(array([ 37,  36,  36, ..., 156, 158, 157], dtype=uint8), 1)

In [136]:
model.predict([X_train[0]])

array([1])

In [137]:
X_test[0], y_test[0]

(array([232, 233, 233, ..., 217, 215, 215], dtype=uint8), 1)

In [138]:
model.predict([X_test[0]])

array([1])

In [144]:
labels = load_cats_dog.target_names

In [147]:

print("actual\t| predicted")
for x,y in list(zip(X_test, y_test))[:4]:
    prediction = model.predict([x])[0]
    
    print(f"{labels[y]}\t| {labels[prediction]}")

actual	| predicted
DOG	| DOG
DOG	| DOG
CAT	| CAT
DOG	| DOG


### Predicting a dog with the image that it never seen before

In [155]:
dog = cv2.imread('dog.jpg', -1)
dog = cv2.resize(cv2.cvtColor(dog, cv2.COLOR_BGR2GRAY), (200, 200))
dog = dog.reshape(-1)
dog

array([147, 111,  53, ..., 141, 125, 125], dtype=uint8)

In [156]:
model.predict([dog])

array([1])

### Evaluating a model

In [133]:
model.score(X_train, y_train)

1.0

In [134]:
model.score(X_test, y_test)

0.66

In [159]:
accuracy_score(y_test, model.predict(X_test))

0.66

In [160]:
accuracy_score(y_train, model.predict(X_train))

1.0

In [165]:
f1_score(y_train, model.predict(X_train), average="binary", labels=["CAT", "DOG"])

1.0

In [166]:
f1_score(y_test, model.predict(X_test), average="binary", labels=["CAT", "DOG"])

0.6666666666666666

## Saving our model

In [173]:
import pickle
with open('./models/cats_dogs.pkl', 'wb') as f:
    pickle.dump(model, f)
    
print("MODEL SAVED")

MODEL SAVED


In [174]:
with open('./models/cats_dogs.pkl', 'rb') as f:
    clf = pickle.load(f)

### Testing our saved model

In [176]:
print("actual\t| predicted")
for x,y in list(zip(X_test, y_test))[:10]:
    prediction = clf.predict([x])[0]
    
    print(f"{labels[y]}\t| {labels[prediction]}")

actual	| predicted
DOG	| DOG
DOG	| DOG
CAT	| CAT
DOG	| DOG
DOG	| DOG
CAT	| CAT
CAT	| DOG
CAT	| CAT
DOG	| CAT
DOG	| DOG
